In [2]:
#| eval: false
/* 导入必要的库 */
proc import 
    datafile = 'C:\Users\asus\Desktop\test\CLHLS\Analysis-0214\clhls_2018_15874.sav'  
    /* 请将这里替换为你的SAV文件的实际路径 */
    out = raw_data 
    dbms = sav 
    replace;
run;

/* 筛选合适的变量并排序 */
data selected_data;
    set raw_data;
    
    /* 因变量 */
    /* 老年人自评健康状况（SHEALTH） */
    SHEALTH = b12;
    
    /* 生活自理能力（ADL） */
    array adl_vars(*) e1 e2 e3 e4 e5 e6;
    array adl_help_days(*) e1b e2b e3b e4b e5b e6b;
    ADL = 0;
    do i = 1 to dim(adl_vars);
        if adl_help_days[i] = 1 then ADL = ADL + adl_vars[i];
        else if adl_help_days[i] = 2 then ADL = ADL + adl_vars[i] * 2;
        else if adl_help_days[i] = 3 then ADL = ADL + adl_vars[i] * 3;
    end;
    
    /* 新增：生活自理障碍二分类变量 ADL2 */
    ADL2 = 0;
    do i = 1 to dim(adl_vars);
        if adl_vars[i] > 1 then do;
            ADL2 = 1;
            leave;
        end;
    end;
    
    /* 工具型生活自理能力（IADL） */
    array iadl_vars(*) e7 e8 e9 e10 e11 e12 e13 e14;
    IADL = 0;
    do i = 1 to dim(iadl_vars);
        IADL = IADL + iadl_vars[i];
    end;
    
    /* 新增：工具型生活自理障碍二分类变量 IADL2 */
    IADL2 = 0;
    do i = 1 to dim(iadl_vars);
        if iadl_vars[i] > 1 then do;
            IADL2 = 1;
            leave;
        end;
    end;
    
    /* 自变量 */
    /* 经济支持 */
    array economic_vars(*) f12a f12b f12c;
    economic_support = 0;
    do i = 1 to dim(economic_vars);
        if economic_vars[i] = 99998 then
            economic_support = economic_support + 10000;
        else if economic_vars[i] in (88888, 99999) then
            continue; /* 跳过无效值或缺失值 */
        else
            economic_support = economic_support + economic_vars[i];
    end;
    
    /* 照料支持 */
    residence = a51;
    living = a52;
    array visit_freq(*) f103a5 f103b5 f103c5 f103d5 f103e5 f103f5 f103g5 f103h5 f103i5 f103j5 f103k5;
    visit_fren = 0;
    do i = 1 to dim(visit_freq);
        if visit_freq[i] = 1 then do;
            visit_fren = 1;
            leave;
        end;
    end;
    
    /* 情感支持 */
    array contact_vars(*) f103a6 f103b6 f103c6 f103d6 f103e6 f103f6 f103g6 f103h6 f103i6 f103j6 f103k6;
    emotion_support = 2; /* 先假设没联系 */
    do i = 1 to dim(contact_vars);
        if contact_vars[i] = 1 then do;
            emotion_support = 1;
            leave;
        end;
    end;
    
    /* 控制变量 */
    /* 年龄 */
    age = trueage;
    
    /* 性别 */
    gender = a1;
    
    /* 受教育程度 */
    education = f1;
    
    /* 退休前的工作类型 */
    job_type = f2;
    
    /* 婚姻状况 */
    if f41 = 1 then marriage_status = 1; /* 假设 1 代表已婚或与伴侣同居 */
    else marriage_status = 0;
    
    /* 户口类型 */
    hukou_type = hukou;
    
    /* 社保和养老保险 */
    if nf64a = 0 or f64b = 1 or f64c = 1 or f64i = 1 then social_insurance = 1;
    else social_insurance = 0;
    
    /* 医疗保险 */
    if f64d = 1 or f64e = 1 or f64g = 1 or f64h = 1 then medical_insurance = 1;
    else medical_insurance = 0;
    
    /* 慢性病 */
    array chronic_vars(*) g15a1 g15b1 g15c1 g15d1 g15e1 g15f1 g15g1 g15h1 g15i1 g15j1 g15k1 g15l1 g15m1 g15n1 g15o1 g15p1 g15q1 g15r1 g15s1 g15t1 g15u1 g15v1 g15w1 g15x1;
    chronic_disease = 0;
    do i = 1 to dim(chronic_vars);
        if chronic_vars[i] = 1 then chronic_disease = 1;
        if chronic_disease = 1 then leave;
    end;
    
    /* 抽烟 */
    smoking = g151;
    
    /* 喝酒 */
    drinking = g161;
    
    /* 体育锻炼 */
    if d91 = 1 or d92 = 1 then exercise = 1;
    else exercise = 2;

    /* 子女年龄状态 */
    array child_ages(*) f103a4 f103b4 f103c4 f103d4 f103e4 f103f4 f103g4 f103h4 f103i4 f103j4 f103k4 f103l4 f103m4;
    array child_alive(*) f103a3 f103b3 f103c3 f103d3 f103e3 f103f3 f103g3 f103h3 f103i3 f103j3 f103k3 f103l3 f103m3;
    last_alive_child_age = .;
    do i = dim(child_ages) to 1 by -1;
        if child_alive[i] = 1 then do;
            last_alive_child_age = child_ages[i];
            leave;
        end;
    end;
    if last_alive_child_age > 60 then child_elderly_status = 1;
    else child_elderly_status = 0;

    /* 生成 care - support 变量 */
    if residence = 1 or visit_fren = 1 then care_support = 1;
    else care_support = 0;
    
    /* 选择需要的变量 */
    keep SHEALTH ADL ADL2 IADL IADL2 economic_support residence living visit_fren emotion_support 
         f10 age gender education job_type marriage_status hukou_type 
         social_insurance medical_insurance chronic_disease smoking drinking exercise
         child_elderly_status care_support f103a3 f103b3 f103c3 f103d3 f103e3 f103f3 f103g3 f103h3 f103i3 f103j3 f103k3 f103l3 f103m3;
run;

/* 将筛选后的保存为 XLSX 格式文件 */
/*
proc export data=selected_data
    outfile='C:\Users\asus\Desktop\test\CLHLS\CLHLS数据\CLHLS数据\clhls_2018_sort0220.xlsx'
    dbms=xlsx
    replace;
run;
*/

/*检查f10 生育子女数的分布情况*/
proc freq data=selected_data;
    tables f10;
run;

/*检查子女存活状态*/
proc freq data=selected_data;
    tables f103a3 f103b3 f103c3 f103d3 f103e3 f103f3 f103g3 f103h3 f103i3 f103j3 f103k3 f103l3 f103m3;
run;

/* 样本筛选 */
data temp_data;
    set selected_data;

    /* 纳入标准 */
    /* 年龄要求 */
    /*age_include = (age > 60);*/
    /* 生育情况 */
    fertility_include = (f10 > 0);

    /* 子女存活情况 */
    array child_alive(*) f103a3 f103b3 f103c3 f103d3 f103e3 f103f3 f103g3 f103h3 f103i3 f103j3 f103k3 f103l3 f103m3;
    child_alive_include = 0;
    do i = 1 to dim(child_alive);
        if child_alive[i] = 1 then do;
            child_alive_include = 1;
            leave;
        end;
    end;
    meet_include = fertility_include and child_alive_include;

    /* 排除标准 */ 
    /* 关键变量缺失值检查 */
    /*array key_vars(*) SHEALTH ADL IADL;*/
    array key_vars(*) SHEALTH ADL ADL2 IADL IADL2 economic_support residence living visit_fren emotion_support 
                      age gender education job_type marriage_status hukou_type 
                      social_insurance medical_insurance chronic_disease smoking drinking exercise
                      child_elderly_status care_support;
    has_missing = 0;
    do i = 1 to dim(key_vars);
        if missing(key_vars[i]) then do;
            has_missing = 1;
            leave;
        end;
    end;
    /* 不合理生育数量检查 */
    unreasonable_fertility = (f10 > 7 or missing(f10));
    meet_exclude = has_missing or unreasonable_fertility;

    /* 筛选符合条件的样本 */
    if meet_include and not meet_exclude;


    /* 移除临时判断变量 */
    drop fertility_include child_alive_include has_missing unreasonable_fertility meet_include meet_exclude;
    /*移除部分原始变量*/
    drop f103a3 f103b3 f103c3 f103d3 f103e3 f103f3 f103g3 f103h3 f103i3 f103j3 f103k3 f103l3 f103m3;
run;

/*打印所有变量的频数分布表，检查是否含有异常值*/
/*
proc freq data=final_data;
    tables _all_;
run;
*/

/*还需要删除含有的样本，即某些变量中赋值为9（not applicable）和88（don't know）的样本*/
/*具体需要剔除变量满足这些条件的样本：SHEALTH>8,ADL>18,IADL>24,residence>3,eudcation>22,smoking>2,drinking>2*/
/* 删除满足特定条件的样本 */
data final_data;
    set temp_data;
    if (SHEALTH <= 8) and (ADL <= 18) and (IADL <= 24) and (residence <= 3) and (age>=60) and (education <= 22) and (smoking <= 2) and (drinking <= 2);
run;

proc freq data=final_data;
    tables _all_;
run;

/* 保存筛选后的数据为 XLSX 格式 */
proc export
    data = final_data
    outfile = 'C:\Users\asus\Desktop\test\CLHLS\Analysis-0214\final_data.xlsx' /* 请替换为实际保存路径 */
    dbms = xlsx
    replace;
run;

/* 对 age 变量进行分组 */
data final_data_grouped;
    set final_data;
    if age < 70 then age_group = '60 - 69';
    else if age < 80 then age_group = '70 - 79';
    else if age < 90 then age_group = '80 - 89';
    else age_group = '90+';
run;

/* 探查每个变量的基本统计信息，查看是否有异常值 */
proc means data=final_data n nmiss min max mean std;
    var SHEALTH ADL ADL2 IADL IADL2 economic_support residence living visit_fren emotion_support
        age gender education job_type marriage_status hukou_type
        social_insurance medical_insurance chronic_disease smoking drinking exercise
        child_elderly_status care_support;
run;

/* 查看字符型变量的唯一值，看是否有异常字符 */
proc freq data=final_data_grouped;
    tables age_group gender education job_type marriage_status hukou_type;
run;

/* 打印因变量、自变量和控制变量的频数分布表并汇总 */
proc freq data=final_data_grouped noprint;
    tables SHEALTH ADL ADL2 IADL IADL2 economic_support residence living visit_fren emotion_support
           age_group gender education job_type marriage_status hukou_type
           social_insurance medical_insurance chronic_disease smoking drinking exercise
           child_elderly_status care_support / out=freq_summary;
run;

/* 导出频数汇总表到 Excel */
proc export
    data = freq_summary
    outfile = 'C:\Users\asus\Desktop\test\CLHLS\Analysis-0214\frequency_summary.xlsx' /* 请替换为实际保存路径 */
    dbms = xlsx
    replace;
run;

5                                                             SAS 系统                                  23:00 Saturday, May 24, 2025

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
NOTE: 正在写入 HTML5(SASPY_INTERNAL) Body（主体）文件: _TOMODS1
25         
26         #| eval: false
           _
           180
ERROR 180-322: 语句无效或未按正确顺序使用。

27         /* 导入必要的库 */
28         proc import
29             datafile = 'C:\Users\asus\Desktop\test\CLHLS\Analysis-0214\clhls_2018_15874.sav'
30             /* 请将这里替换为你的SAV文件的实际路径 */
31             out = raw_data
32             dbms = sav
33             replace;

34         run;
35         
36         /* 筛选合适的变量并排序 */
37         data selected_data;
38             set raw_data;
ERROR: 文件“WORK.RAW_DATA.DATA”不存在。
39         
40             /* 因变量 */
41             /* 老年人自评健康状况（SHEALTH） */
42             SHEALTH = b12;
43         
44      

ERROR 180-322: 语句无效或未按正确顺序使用。 None


In [3]:
your SAS code here 

7                                                             SAS 系统                                  23:00 Saturday, May 24, 2025

323        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
323      ! ods graphics on / outputfmt=png;
NOTE: 正在写入 HTML5(SASPY_INTERNAL) Body（主体）文件: _TOMODS1
324        
325        your SAS code here
326        
327        
328        ods html5 (id=saspy_internal) close;ods listing;
329        
8                                                             SAS 系统                                  23:00 Saturday, May 24, 2025

330        


In [4]:
#| eval: false 
your code here 

9                                                             SAS 系统                                  23:00 Saturday, May 24, 2025

333        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
333      ! ods graphics on / outputfmt=png;
NOTE: 正在写入 HTML5(SASPY_INTERNAL) Body（主体）文件: _TOMODS1
334        
335        #| eval: false
336        your code here
337        
338        
339        ods html5 (id=saspy_internal) close;ods listing;
340        
10                                                            SAS 系统                                  23:00 Saturday, May 24, 2025

341        
